## 모두를 위한 딥러닝 Lab#04-2
#### 2018.06.09

---
- Loading Data From File

In [1]:
import numpy as np

xy = np.loadtxt('test04-2.csv', delimiter=',', dtype=np.float32)

x_data = xy[:,0:-1]
y_data = xy[:,-1]

print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data, len(y_data))

(6, 3) [[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]
 [ 53.  46.  55.]] 6
(6,) [152. 185. 180. 196. 142. 101.] 6


In [2]:
import numpy as np
import tensorflow as tf

# 파일 로드
xy = np.loadtxt('test04-2.csv', delimiter=',', dtype=np.float32)

# 인풋 : 마지막 이전 열 까지

# 아웃풋 : 마지막 열 까지
x_data = xy[:,0:-1]
y_data = xy[:,[-1]]

# 홀더
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(10001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    
    if i % 500 == 0:
        print(i, cost_val, hy_val)


0 61875.61 [[ -83.5367  ]
 [ -96.81211 ]
 [ -97.532   ]
 [-102.14741 ]
 [ -76.39897 ]
 [ -49.811474]]
500 13.038539 [[150.19923]
 [183.75476]
 [179.11546]
 [198.9778 ]
 [137.62453]
 [107.68157]]
1000 12.31618 [[150.43712]
 [183.66304]
 [179.23155]
 [198.96233]
 [137.57735]
 [107.38289]]
1500 11.69263 [[150.63988 ]
 [183.58806 ]
 [179.33307 ]
 [198.93886 ]
 [137.55118 ]
 [107.118706]]
2000 11.143941 [[150.81287]
 [183.52722]
 [179.42221]
 [198.9088 ]
 [137.54251]
 [106.88397]]
2500 10.652697 [[150.9606 ]
 [183.4783 ]
 [179.50082]
 [198.87337]
 [137.54843]
 [106.67436]]
3000 10.206315 [[151.08693]
 [183.43942]
 [179.57043]
 [198.83354]
 [137.56645]
 [106.48625]]
3500 9.79563 [[151.1951 ]
 [183.40897]
 [179.63235]
 [198.79019]
 [137.59447]
 [106.31653]]
4000 9.413924 [[151.2879 ]
 [183.38567]
 [179.68776]
 [198.74406]
 [137.63072]
 [106.1626 ]]
4500 9.056306 [[151.3676 ]
 [183.3683 ]
 [179.73752]
 [198.69571]
 [137.67366]
 [106.0222 ]]
5000 8.719027 [[151.43626 ]
 [183.35599 ]
 [179.78252

- 예측

In [17]:
print(sess.run(hypothesis, feed_dict={X:[[60, 70, 110], [100, 100, 100], [10, 0, 0]]}))

[[151.45012 ]
 [201.20299 ]
 [  7.885025]]


- Queue Runners
    
> 파일 여러 개 리딩시 시간이 오래 걸릴 때 큐를 사용한다


In [13]:
import tensorflow as tf

filename_queue = tf.train.string_input_producer(['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

record_default = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_default)

train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=11)

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1]) 
                                                
W = tf.Variable(tf.random_normal([3, 1]), name='weight')                                      
b = tf.Variable(tf.random_normal([1]), name='bias')                                      
                                            
hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)

train = optimizer.minimize(cost)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

coord = tf.train.Coordinator()

threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for i in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    print(x_batch, y_batch)
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    
    if i % 100 == 0 :
        print(i, '\t', cost_val, '\t', hy_val)
        
coord.request_stop()
coord.join(threads)
                                                

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Field 0 in record 0 is not a valid float: # EXAM1
	 [[Node: DecodeCSV = DecodeCSV[OUT_TYPE=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], field_delim=",", na_value="", use_quote_delim=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ReaderReadV2:1, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0, DecodeCSV/record_defaults_0)]]


OutOfRangeError: FIFOQueue '_196_batch_10/fifo_queue' is closed and has insufficient elements (requested 10, current size 0)
	 [[Node: batch_10 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_FLOAT], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](batch_10/fifo_queue, batch_10/n)]]

Caused by op 'batch_10', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Version